In [ ]:
from itertools import izip
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
cd ~/g/projects/black_spruce/

In [ ]:
all_graphs = !ls *graph*all*.txt
needle_graphs = !ls *graph*needle*.txt
cambium_graphs = !ls *graph*cambium*.txt


In [ ]:
graphs = {"All":all_graphs,
         "Needle":needle_graphs,
         "Cambium":cambium_graphs}

In [ ]:
def plot_pie(plot_data, title):
    unique_seqs = set()
    
    fig = plt.gcf()
    fig.set_size_inches(8,8)
    plot_data = plot_data.ix[0:20,["Term", "#Seqs","Sequences"]]
    plot_data['Label'] = plot_data.apply(lambda row: "%s (%d)" % (row["Term"],row["#Seqs"]), axis=1)
    plt.pie(plot_data["#Seqs"],
           colors=sns.color_palette("hls", len(plot_data)),
           labels = plot_data["Label"])
    
    for row in plot_data.iterrows():
        for seq in row[1].Sequences.split(","):
            unique_seqs.add(seq)
    
    plt.title("%s (%d)"  % (title, len(unique_seqs)))

    plt.show()

In [ ]:
plotted = 0
for key, files in graphs.items():
    print "================= Charts for %s =================" % key
    for g in files:
        ontology = g[-6:-4].upper()
        df = pd.read_csv(g, sep="\t", header=0, index_col=1)
        for level, data in df.groupby("Level"):
            if level > 1:
                d = data.sort("#Seqs", ascending=False)
                plot_pie(d, "%s/%s/Level %d" % (key, ontology, level))
                plotted += 1

In [ ]:
plotted